# Explore and Run Shot Detection

Running facial regonition and embedding on every single frame is expensive. Not to mention, having to query a database or even store information for every frame would get dense.

Can we use shot detection/boundaries to determine when sequential frames show significant variation? If yes, we can run facial detection for key frames from these shots, which will *hopefully* contain all individuals appearing in a given shot.

## Process MP4 File 

In order to perform shot/scene detection we need to attach some information that will help us quantify the content appearing in a frame 

Using these metrics we can calculate when a significant shift occurs between frames and mark it as a shot boundary

In [1]:
import sys 

sys.path.append("/Users/srmarshall/Desktop/code/pbs/pbs-passthrough/")

In [2]:
# set mp4 filepath - in this case a path pointing to a grantchester episode 
mp4_filepath = "/Users/srmarshall/Desktop/data-dump/dynamic_recaps/video_assets/full_length/30d5ccbd-f2ce-4fd7-99d9-ee28236d9af9.mp4"

In [3]:
from utils.helpers import video_procesisng_pipeline

# run video processing pipeline
df = video_procesisng_pipeline(mp4_filepath=mp4_filepath, load_previous=True)

Extracting Features:   0%|          | 16/95025 [00:00<10:19, 153.24 frames/s]

Error extracting features: cannot access local variable 'edge' where it is not associated with a value


Extracting Features: 95026 frames [10:15, 154.34 frames/s]                      


In [4]:
## write to df to avoid reprocessing 
# df.to_csv("../assets/grantchester_sample_features.csv", index=False) ## SM: only needs to be done once

In [5]:
df.head()

,frame_number,timestamp,edges,pixel_diffs,bhattacharyya_distance
0,1,0.000000,0,0.0,0.0
1,2,33.366667,0,0.0,0.0
2,3,66.733333,0,0.0,0.0
3,4,100.100000,0,0.0,0.0
4,5,133.466667,0,0.0,0.0


In [6]:
df.tail()

,frame_number,timestamp,edges,pixel_diffs,bhattacharyya_distance
95020,95021,3.170501e+06,0,0.0,0.0
95021,95022,3.170534e+06,0,0.0,0.0
95022,95023,3.170567e+06,0,0.0,0.0
95023,95024,3.170601e+06,0,0.0,0.0
95024,95025,3.170634e+06,0,NaN,NaN


## Detect Key Frames

Use the metrics we have available to us to try and determine when a frame marks a shot boundary 

For now, let's use the `bhattacharyya_distance`

In [11]:
# summary stats
df.describe()

,frame_number,timestamp,edges,pixel_diffs,bhattacharyya_distance
count,95025.000000,9.502500e+04,95025.000000,95024.000000,93983.000000
mean,47513.000000,1.585317e+06,12918.538374,18438.002220,0.021828
std,27431.499002,9.152977e+05,16596.805285,56571.436779,0.040937
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,23757.000000,7.926585e+05,4045.000000,3.000000,0.009606
50%,47513.000000,1.585317e+06,7504.000000,1522.000000,0.013762
75%,71269.000000,2.377976e+06,14369.000000,12630.250000,0.020771
max,95025.000000,3.170634e+06,143585.000000,830705.000000,0.994448


In [12]:
# pull mean and std 
mean = df["bhattacharyya_distance"].mean()
std = df["bhattacharyya_distance"].std()

# set threshold to 2 std above the mean
threshold = mean + 2*std

In [13]:
import numpy as np 

# mark shot boundaries using the threshold 
df["is_keyframe"] = np.where(df["bhattacharyya_distance"] > threshold, 1, 0)

In [14]:
# how many shots do we have?
df.value_counts("is_keyframe")

is_keyframe
0    93307
1     1718
Name: count, dtype: int64

In [15]:
# extract keyframes for analysis
keyframe_df = df[df["is_keyframe"] == 1]

## Embed Key Frames